In [85]:
import os
import easyocr
import numpy as np
import re
import pandas as pd
from PIL import Image
import mysql.connector as sql


In [150]:
reader = easyocr.Reader(['en'],gpu = False)

Using CPU. Note: This module is much faster with a GPU.


In [153]:
im = Image.open(r"F:\VS Code\4.png")
im_arr= np.array(im)
text = reader.readtext(im_arr,detail= 0)
text



['REVANTH',
 'Marketing Executive',
 '123 ABC St,, HYDRABAD, TamilNadu;',
 '600001',
 '+91-456-1234',
 'hello@CHRISTMAS.com',
 'Family',
 'wwW.CHRISTMAS.com',
 'Restaurant']

In [154]:
def extract_data(data):
    result = {"NAME": [], "DESIGNATION": [], "ADDRESS": [], "COMPANY_NAME": [], "CONTACT": [], "EMAIL": [], "WEBSITE": [], "PINCODE": []}


    result["NAME"].append(data[0])
    result["DESIGNATION"].append(data[1])

    for item in range(2, len(data)):

        if "@"  in data[item] and ".com" in data[item]:
            result["EMAIL"].append(data[item].lower())

        elif 'www' in data[item] or 'WWW' in data[item] or 'wwW' in data[item]  and ".com" in data[item]:
            result["WEBSITE"].append(data[item].lower())

        elif "Tamil Nadu" in data[item] or "TamilNadu" in data[item]  or re.match(r'\b\d{6}\b', data[item])  or data[item].isdigit():
            result["PINCODE"].append(data[item])

        elif re.match(r"^[a-zA-Z\s,]", data[item]):
            result["COMPANY_NAME"].append(data[item])

        elif data[item].startswith("+") or (data[item].replace("-", "").isdigit()):
            result["CONTACT"].append(data[item])
            
        else:
            filtered_add = re.sub(r"[,;]", "", data[item])
            result["ADDRESS"].append(filtered_add)

    for key, value in result.items():
        if len(value)>0:
            something = ''.join(value)
            result[key] = [something]

        else:
            value = 'None'
            result[key] = [value]


            
    return result
extract_data(text)

{'NAME': ['REVANTH'],
 'DESIGNATION': ['Marketing Executive'],
 'ADDRESS': ['None'],
 'COMPANY_NAME': ['FamilyRestaurant'],
 'CONTACT': ['+91-456-1234'],
 'EMAIL': ['hello@christmas.com'],
 'WEBSITE': ['www.christmas.com'],
 'PINCODE': ['123 ABC St,, HYDRABAD, TamilNadu;600001']}

In [131]:
my_db = sql.connect( 
             host="127.0.0.1",
             user="root",
             database = "BizcardX",
             port = 3306,
             password="2369",
              ) 
      
mycursor = my_db.cursor() 


In [132]:
def convert_sql(x):
    table = ''' CREATE TABLE IF NOT EXISTS Card_details ( Name VARCHAR(225) PRIMARY KEY,
                                                          Designation VARCHAR(225),
                                                          Address VARCHAR(225),
                                                          Company_Name VARCHAR(225),
                                                          Contact VARCHAR(225),
                                                          Email VARCHAR(225),
                                                          Website VARCHAR(225),
                                                          Pincode VARCHAR(225) )'''
    mycursor.execute(table)
    my_db.commit()

    df = pd.DataFrame(x)

    insert_data = ''' INSERT INTO Card_details ( Name , Designation, Address, Company_Name, Contact, Email, Website, Pincode)
                                               VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'''
    

    for index, row in df.iterrows():
        values = tuple(row)
        mycursor.execute(insert_data, values)
        my_db.commit()

    

In [133]:
details = extract_data(text)
convert_sql(details)

In [138]:
def delete_table(x):
    mycursor.execute("DELETE FROM Card_details WHERE Name = %s ", (x,))
    table = mycursor.fetchall()
    my_db.commit()

    mycursor.execute("SELECT * FROM Card_details")
    table1 = mycursor.fetchall()

    df = pd.DataFrame(table1, columns = mycursor.column_names)

    my_db.commit()

    return df



In [139]:
def show_table(x):
    mycursor.execute("SELECT * FROM Card_details")
    table = mycursor.fetchall()

    df = pd.DataFrame(table, columns = mycursor.column_names)

    my_db.commit()

    return df

,Name,Designation,Address,Company_Name,Contact,Email,Website,Pincode
0,SANTHOSH,Technical Manager,None,Sun Electricals,+123-456-1234,hello@sun.com,www.suncom,"123 ABC St , Tirupur, TamilNadu;641603"
1,Selva,DATA MANAGER,123 ABC St Chennai,selvadigitals,+123-456-7890+123-456-7891,hello@xyz1.com,www xyzi.com,TamilNadu 600113
2,Amit kumar,CEO & FOUNDER,123 global,"global.comErode,GLOBALINSURANCESt ,",123-456-7569,hello@global.com,www,TamilNadu 600115
